# Simple Convnet - MNIST

Slightly modified from mnist_cnn.py in the Keras examples folder:

**https://github.com/fchollet/keras/blob/master/examples/mnist_cnn.py**

In [1]:
WEIGHTS_FILEPATH = 'mnist_cnn.hdf5'
MODEL_ARCH_FILEPATH = 'mnist_cnn.json'

In [2]:
import numpy as np
np.random.seed(1337)  # for reproducibility

from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Convolution2D, MaxPooling2D
from keras.utils import np_utils
from keras.callbacks import EarlyStopping, ModelCheckpoint

Using TensorFlow backend.


In [3]:
nb_classes = 10

# input image dimensions
img_rows, img_cols = 28, 28

# the data, shuffled and split between train and test sets
(X_train, y_train), (X_test, y_test) = mnist.load_data()
X_train = X_train.reshape(X_train.shape[0], img_rows, img_cols, 1)
X_test = X_test.reshape(X_test.shape[0], img_rows, img_cols, 1)
input_shape = (img_rows, img_cols, 1)

X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train /= 255
X_test /= 255
print('X_train shape:', X_train.shape)
print(X_train.shape[0], 'train samples')
print(X_test.shape[0], 'test samples')

# convert class vectors to binary class matrices
Y_train = np_utils.to_categorical(y_train, nb_classes)
Y_test = np_utils.to_categorical(y_test, nb_classes)

X_train shape: (60000, 28, 28, 1)
60000 train samples
10000 test samples


In [4]:
# Sequential Model

model = Sequential()
model.add(Convolution2D(32, 3, 3, border_mode='valid', input_shape=input_shape, dim_ordering='tf'))
model.add(Activation('relu'))
model.add(Convolution2D(32, 3, 3, border_mode='valid', dim_ordering='tf'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2), border_mode='valid', dim_ordering='tf'))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(nb_classes))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [5]:
# Model saving callback
checkpointer = ModelCheckpoint(filepath=WEIGHTS_FILEPATH, monitor='val_acc', verbose=1, save_best_only=True)

# Early stopping
early_stopping = EarlyStopping(monitor='val_acc', verbose=1, patience=5)

# Train
batch_size = 128
nb_epoch = 100
model.fit(X_train, Y_train, batch_size=batch_size, nb_epoch=nb_epoch, verbose=2,
          callbacks=[checkpointer, early_stopping], 
          validation_data=(X_test, Y_test))
score = model.evaluate(X_test, Y_test, verbose=0)
print('Test score:', score[0])
print('Test accuracy:', score[1])

Train on 60000 samples, validate on 10000 samples
Epoch 1/100
Epoch 00000: val_acc improved from -inf to 0.98090, saving model to mnist_cnn.hdf5
12s - loss: 0.2862 - acc: 0.9141 - val_loss: 0.0603 - val_acc: 0.9809
Epoch 2/100
Epoch 00001: val_acc improved from 0.98090 to 0.98600, saving model to mnist_cnn.hdf5
11s - loss: 0.1023 - acc: 0.9694 - val_loss: 0.0407 - val_acc: 0.9860
Epoch 3/100
Epoch 00002: val_acc improved from 0.98600 to 0.98940, saving model to mnist_cnn.hdf5
11s - loss: 0.0749 - acc: 0.9782 - val_loss: 0.0339 - val_acc: 0.9894
Epoch 4/100
Epoch 00003: val_acc did not improve
11s - loss: 0.0603 - acc: 0.9815 - val_loss: 0.0308 - val_acc: 0.9888
Epoch 5/100
Epoch 00004: val_acc improved from 0.98940 to 0.99080, saving model to mnist_cnn.hdf5
11s - loss: 0.0535 - acc: 0.9836 - val_loss: 0.0297 - val_acc: 0.9908
Epoch 6/100
Epoch 00005: val_acc improved from 0.99080 to 0.99150, saving model to mnist_cnn.hdf5
11s - loss: 0.0481 - acc: 0.9853 - val_loss: 0.0270 - val_acc: 0

In [6]:
with open(MODEL_ARCH_FILEPATH, 'w') as f:
    f.write(model.to_json())